In [1]:
import cv2
import numpy as np

In [2]:
cv2.namedWindow('ObjectDetection', cv2.WINDOW_AUTOSIZE)

path = './motor_bike.mp4'
cap = cv2.VideoCapture(path)

classNames = {0: 'background',
              1: 'aeroplane',
              2: 'bicycle',
              3: 'bird',
              4: 'boat',
              5: 'bottle',
              6: 'bus',
              7: 'car',
              8: 'cat',
              9: 'chair',
              10: 'cow',
              11: 'diningtable',
              12: 'dog',
              13: 'horse',
              14: 'motorbike',
              15: 'person',
              16: 'pottedplant',
              17: 'sheep',
              18: 'sofa',
              19: 'train',
              20: 'tvmonitor'}

net = cv2.dnn.readNetFromCaffe('./MobileNetSSD_deploy.prototxt', './MobileNetSSD_deploy.caffemodel')
conf_thresh = 0.7

if not cap.isOpened():
    print('File not ready')
    
while True:
    
    retval, frame = cap.read()
    if not retval or cv2.waitKey(1) == ord('q'):
        break
    else:
        frame = cv2.resize(frame, dsize= (300, 300))
        height, width, _ = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), mean= [127.5, 127.5, 127.5], swapRB= False)
        
        net.setInput(blob)
        preds = net.forward()
        
        for i in range(preds.shape[2]):
            
            label_id = preds[0, 0, i, 1]
            confidence = round(preds[0, 0, i, 2], 2)
            
            if confidence > conf_thresh:
                
                top, left, bottom, right = preds[0, 0, i, 3:]
                
                top_coord, left_coord = int(top * width), int(left * height)
                bottom_coord, right_coord = int(bottom * width), int(right * height)
                
                colors = np.random.uniform(0, 255, (preds.shape[2], 3))
                cv2.rectangle(frame, (top_coord, left_coord), (bottom_coord, right_coord), [int(arg) for arg in colors[i]])
                cv2.putText(frame, classNames[label_id], (top_coord + 10, left_coord - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, [int(arg) for arg in colors[i]])
                
        cv2.imshow('ObjectDetection', frame)
cap.release()
cv2.destroyAllWindows()